# Controlabilidade e Observabilidade

Supondo um SLIT, o código permite avaliar se é ou não controlável a partir das matrizes A e B, avaliando o gramiano, tal qual utilizando as matrizes A e C, avaliando a observabilidade.

In [1]:
import numpy as np
import control

As duas funções que realizam a análise de controlabilidade e observabilidade do sistema

In [2]:
def is_controllable(A, B):
    """
    Check if a system defined by its state matrix A and input matrix B is controllable.
    """
    sys = control.StateSpace(A, B, np.eye(A.shape[0]), np.zeros((A.shape[0], B.shape[1])))
    return control.ctrb(sys.A, sys.B).shape[0] == A.shape[0]

In [3]:
def is_observable(A, C):
    """
    Check if a system defined by its state matrix A and output matrix C is observable.
    """
    sys = control.StateSpace(A, np.zeros((A.shape[0], 1)), C, np.zeros((C.shape[0], 1)))
    return control.obsv(sys.A, sys.C).shape[0] == A.shape[0]

Abaixo, as matrizes do sistema definidas

In [4]:
Xcm = 1.052
Xpe = 0.261
L = 3.304
Df = 0.127
Tlin = 230.57
Izz = 15.07
M = 23.545
g = 9.8055
dar = 1.091
Cd = 0.432
Vxlin = -10.0

In [5]:
# X = [x,y,theta,xp,yp,thetap]
# Xp = [xp,yp,thetap,xpp,ypp,thetapp]
A = np.array([[0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1],
              [0, 0, 0, (-1/M)*(Vxlin *dar*Cd*((Df/2)**2*np.pi)), 0, 0],
              [0, 0, (1/M)*(-M*g-(1/2)*Vxlin**2*dar*Cd*(np.pi*(Df/2)**2)), 0, 0, 0],
              [0, 0, (1/Izz)*((1/2)*Vxlin**2*dar*Xpe*Cd*(np.pi*(Df/2)**2)), 0, 0, 0]])
# u = [T,tau]
B = np.array([[0, 0],
              [0, 0],
              [0, 0],
              [1/M, 0],
              [0, -Tlin/M],
              [0, -Tlin*Xcm/Izz]])

C = np.array([[ 0, 0, 1, 0, 0, 0],
              [ 0, 0, 0, 1, 0, 0],
              [ 0, 0, 0, 0, 1, 0]])

D = np.array([[ 0, 0],
              [ 0, 0],
              [ 0, 0]])

In [6]:
if is_controllable(A, B):
    print("System is controllable.")
else:
    print("System is not controllable.")

if is_observable(A, C):
    print("System is observable.")
else:
    print("System is not observable.")

System is controllable.
System is not observable.
